# Ray RLlib - Sample Application: MountainCar-v0

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `MountainCar-v0` environment, ([gym.openai.com/envs/MountainCar-v0/](https://gym.openai.com/envs/MountainCar-v0/))

For more background about this problem, see:

* ["Efficient memory-based learning for robot control"](https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-209.pdf), [Andrew William Moore](https://www.cl.cam.ac.uk/~awm22/), University of Cambridge (1990)
* ["Solving Mountain Car with Q-Learning"](https://medium.com/@ts1829/solving-mountain-car-with-q-learning-b77bf71b1de2), [Tim Sullivan](https://twitter.com/ts_1829)

Import Ray and the PPO support, then start Ray…

In [1]:
import ray
import ray.rllib.agents.ppo as ppo

Let's start up Ray as in the previous lessons:

In [ ]:
!../tools/start-ray.sh --check --verbose

In [2]:
ray.init(ignore_reinit_error=True)

2020-05-01 12:19:34,512	INFO resource_spec.py:212 -- Starting Ray with 4.25 GiB memory available for workers and up to 2.14 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:19:34,848	INFO services.py:1148 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:52328',
 'object_store_address': '/tmp/ray/session_2020-05-01_12-19-34_501646_27188/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-01_12-19-34_501646_27188/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-05-01_12-19-34_501646_27188'}

The Ray Dashboard is useful for monitoring Ray:

In [3]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8266


Next we'll train an RLlib policy with the `MountainCar-v0` environment.

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to train longer and see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/mountain-car` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [4]:
SELECT_ENV = "MountainCar-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=select_env)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo/mountain-car")
    print(f'\ncheckpoint saved to {file_name}')

2020-05-01 12:22:58,588	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:22:58,622	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-05-01 12:23:00,735	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:23:00,736	WARNING util.py:37 -- Install gputil for GPU system monitoring.


{'episode_reward_max': -200.0, 'episode_reward_min': -200.0, 'episode_reward_mean': -200.0, 'episode_len_mean': 200.0, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0], 'episode_lengths': [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]}, 'sampler_perf': {'mean_env_wait_ms': 0.09571272751380658, 'mean_processing_ms': 0.09866287444961601, 'mean_inference_ms': 0.5515321143444392}, 'off_policy_estimator': {}, 'info': {'num_steps_trained': 3968, 'num_steps_sampled': 4000, 'sample_time_ms': 3499.21, 'load_time_ms': 51.573, 'grad_time_ms': 1879.769, 'update_time_ms': 505.508, 'learner': {'default_policy': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 4.999999873689376e-05, '

In [5]:
print(reward_history)

[-200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0, -200.0]


Do the episode rewards increase after multiple iterations?
That shows whether the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [6]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(2, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(2, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 3) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(3,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
__________

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "car" agent operating within the simulation: rocking back and forth to gain momentum to overcome the mountain.

In [7]:
! rllib rollout \
    /tmp/ppo/mountain-car/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"MountainCar-v0\"}" --run PPO \
    --steps 2000

2020-05-01 12:24:29,927	INFO resource_spec.py:212 -- Starting Ray with 4.25 GiB memory available for workers and up to 2.13 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:24:30,253	INFO services.py:1148 -- View the Ray dashboard at localhost:8267
2020-05-01 12:24:30,901	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:24:30,932	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-05-01 12:24:33,699	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:24:33,700	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-01 12:24:33,768	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:24:33,768	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /tmp/ppo/mountain-car/checkpoint_2/checkpoint-2
2020-05-01 12:24:33,768	INF

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (copy/paste the URL it generates) to visualize key metrics from training with RLlib…

In [ ]:
!tensorboard --logdir=~/ray_results/

Next, go through any of other remaining lessons:

* [04b: Application: Taxi](04b-Application-Taxi.ipynb) -- Based on the `Taxi-v3` environment from OpenAI Gym.
* [04c: Application: Frozen Lake](04c-Application-Frozen-Lake.ipynb) -- Based on the `FrozenLake-v0` environment from OpenAI Gym.